### Prompt Compression using LLMLingua Python library and OpenAI

In [1]:
!pip install -q llmlingua openai accelerate python-dotenv llama-index llama-index-postprocessor-longllmlingua

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.7/226.7 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 19.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.9/75.9 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 631.0/631.0 kB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 41.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.0/284.0 kB 36.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.6/30.6 MB 52.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 6.5 MB/s eta 0:00:00
ERROR: pip's dependency resolv

In [1]:
#Google drive mount
from google.colab import drive
drive.mount('/content/drive/')
%cd drive/MyDrive/Upwork_Projects/

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive/Upwork_Projects


In [2]:
#Env variable for openai api key
from dotenv import load_dotenv
import os
import openai
load_dotenv()
api_key = os.getenv("openai_api_key")
openai.api_key = api_key

In [3]:
#Load pdf documents
from llama_index.core import download_loader, VectorStoreIndex
PDFReader = download_loader("PDFReader")
loader = PDFReader()
documents = loader.load_data(file='data/Power of The Subconscious Mind - Joseph Murphy.pdf')
print(type(documents))
print(len(documents))

<ipython-input-3-ae2831fa17a4>:2: DeprecationWarning: Call to deprecated function (or staticmethod) download_loader. (`download_loader()` is deprecated. Please install tool using pip install directly instead.)
  PDFReader = download_loader("PDFReader")


<class 'list'>
222


In [4]:
#LLama-index vector store
index = VectorStoreIndex.from_documents(documents)
retriever = index.as_retriever(similarity_top_k=4)

In [5]:
#Question and contexts
question = "What is the difference between conscious mind and subconscious mind?"
contexts = retriever.retrieve(question)

In [ ]:
context_list = [n.get_content() for n in contexts]
context_list[0]

In [7]:
#Prompt Compressor
from llmlingua import PromptCompressor
llm_lingua = PromptCompressor(
    model_name="NousResearch/Llama-2-7b-hf",  # Default model
    device_map="cuda",  # Device environment (e.g., 'cuda', 'cpu', 'mps')
    model_config={},  # Configuration for the Huggingface model
    open_api_config={},  # Configuration for OpenAI Embedding
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [8]:
compressed_prompt = llm_lingua.compress_prompt(context_list,
                                               instruction="Given the context, please answer the question correctly.",
                                               question=question,
                                               ratio = 0.7,
                                               target_token=-1,
                                               rank_method= "llmlingua")

In [ ]:
print(compressed_prompt['compressed_prompt'])

In [10]:
# The response from original and compressed prompt
from llama_index.llms.openai import OpenAI
import tiktoken
# Combine prompts
original_prompt = "\n\n".join(context_list + [question])
compressed_prompt_text = compressed_prompt['compressed_prompt']
prompts = [original_prompt, compressed_prompt_text]
# Get the encoding object
enc = tiktoken.get_encoding("cl100k_base")
# Tokenize prompts
tokenized_prompts = [(p, len(enc.encode(p))) for p in prompts]
# Calculate compressed ratio
compressed_ratio = tokenized_prompts[0][1] / (tokenized_prompts[1][1] + 1e-5)
# Format output strings
token_comp = f"Original Prompt: {tokenized_prompts[0][1]}\n" + \
             f"Compressed Prompt: {tokenized_prompts[1][1]}\n" + \
             f"Compressed Ratio: {compressed_ratio:.2f}x"
print(token_comp)
print("\n")
# Initialize OpenAI model
llm = OpenAI(model="gpt-3.5-turbo-0125")
# Batch processing of prompts
for prompt_text, _ in tokenized_prompts:
    response = llm.complete(prompt_text)
    print(str(response))
    print("-" * 100)

Original Prompt: 1796
Compressed Prompt: 532
Compressed Ratio: 3.38x


The conscious mind is the reasoning mind, which makes decisions and choices based on conscious thought. It is the part of the mind that is aware of the present moment and actively engages in decision-making. On the other hand, the subconscious mind operates below the level of conscious awareness and is responsible for automatic functions such as breathing, digestion, and circulation. It does not reason or argue like the conscious mind, but rather accepts and acts upon the beliefs and suggestions that are impressed upon it. The subconscious mind is like a soil that accepts any kind of seed, whether positive or negative, and brings about corresponding results in the outer experience. The conscious and subconscious minds are not separate entities, but rather two spheres of activity within one mind.
----------------------------------------------------------------------------------------------------
The conscious mind is

### Prompt Compression using Llama-Index's longllmlingua

In [7]:
# Import the necessary modules from llama_index
# from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.core.response_synthesizers import CompactAndRefine
from llama_index.postprocessor.longllmlingua import LongLLMLinguaPostprocessor
# Create a postprocessor object using LongLLMLinguaPostprocessor with the specified parameters
node_postprocessor = LongLLMLinguaPostprocessor(
    instruction_str="Given the context, please answer the final question",
    target_token=300,
    rank_method="longllmlingua",
    additional_compress_kwargs={
        "condition_compare": True,
        "condition_in_question": "after",
        "context_budget": "+100",
        "reorder_context": "sort",  # enable document reorder,
        "dynamic_context_compression_ratio": 0.5,
    },
)
# Create a retriever object with the default parameters
# The retriever will use the vector store index to fetch the most relevant nodes
retrieved_nodes = retriever.retrieve(question)
# Create a synthesizer object with the default parameters
# The synthesizer will use the compact and refine mode to generate a response
synthesizer = CompactAndRefine()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:386: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.6` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`. This was detected when initializing the generation config instance, which means the corresponding file may hold incorrect parameterization and should be fixed.
  warnings.warn(


In [8]:
from llama_index.core.indices.query.schema import QueryBundle
new_retrieved_nodes = node_postprocessor.postprocess_nodes(
    retrieved_nodes, query_bundle=QueryBundle(query_str=question)
)

In [9]:
#Count original and compressed tokens
original_contexts = "\n\n".join([n.get_content() for n in retrieved_nodes])
compressed_contexts = "\n\n".join([n.get_content() for n in new_retrieved_nodes])

original_tokens = node_postprocessor._llm_lingua.get_token_length(original_contexts)
compressed_tokens = node_postprocessor._llm_lingua.get_token_length(compressed_contexts)

# print(compressed_contexts)
print("\n")
print("Original Tokens:", original_tokens)
print("Compressed Tokens:", compressed_tokens)
print("Comressed Ratio:", f"{original_tokens/(compressed_tokens + 1e-5):.2f}x")



Original Tokens: 2110
Compressed Tokens: 83
Comressed Ratio: 25.42x


In [10]:
#llama-index synthesizer for response generation
response = synthesizer.synthesize(question, new_retrieved_nodes)

In [11]:
print(str(response))

The conscious mind and the subconscious mind are two different aspects of our mental processes. The conscious mind is responsible for our awareness and rational thinking. It is the part of our mind that we actively use to make decisions, solve problems, and engage in logical reasoning. On the other hand, the subconscious mind operates below the level of conscious awareness. It is responsible for storing and processing information that we are not actively thinking about. The subconscious mind influences our thoughts, emotions, and behaviors, often without us realizing it. It is also believed to play a role in shaping our beliefs, habits, and automatic responses.
